# Chapter 3.5
## Reduced Animal Model
Zigui Wang

Consider the following data set for the pre-weaning gain (WWG) of beef calves (calves assumed to be reared 
under the same management conditions). The objective is to estimate the effects of sex and predictive breeding values for all animals. The data set is obtained from the example 3.1

Assume $\sigma^2_a = 20$ and $\sigma^2_e = 40$, $\alpha = \frac{\sigma^2_e}{\sigma^2_a}$


In [7]:
using JWAS, DataFrames,LinearAlgebra

In [48]:
invA = [
     1.833  0.500  0.000 -0.667  0.000 -1.000  
     0.500  2.000  0.500  0.000 -1.000 -1.000 
     0.000  0.500  1.500  0.000 -1.000  0.000  
    -0.667  0.000  0.000  1.333  0.000  0.000 
     0.000 -1.000 -1.000  0.000  2.000  0.000 
    -1.000 -1.000  0.000  0.000  0.000  2.000 
];

In [3]:
data = DataFrame(Calves = [4,5,6,7,8]
    ,sex=["male","female","female","male","male"], 
    sire=[1,3,1,4,3], 
    Dam=[missing,2,2,5,6],
    WWG=[4.5,2.9,3.9,3.5,5.0])

,Calves,sex,sire,Dam,WWG
,Int64,String,Int64,Int64⍰,Float64
1,4,male,1,missing,4.5
2,5,female,3,2,2.9
3,6,female,1,2,3.9
4,7,male,4,5,3.5
5,8,male,3,6,5.0


### Step 1: Constructing the R

In our case, since calves 4, 5 and 6 are parents, the diagonal elements corresponding to these three are equal to the $\sigma^2_e$, which is 40.

Since calves 7 and 8 are non parents. The diagonal elements corresponding to these two are equal to the $\sigma^2_e + d_i \sigma^2_a$. For each calf, $d_i$ equal to $\frac{1}{2}$ since both their parents are known. Therefore, $r_{77} = r_{88} = \sigma^2_e + d \alpha^{-1} \sigma^2_e = 40 + \frac{1}{2}*\frac{1}{2}*40 = 50 $

In [9]:
R = [
    40 0 0 0 0
    0 40 0 0 0
    0 0 40 0 0
    0 0 0 50 0
    0 0 0 0 50
]

5×5 Array{Int64,2}:
 40   0   0   0   0
  0  40   0   0   0
  0   0  40   0   0
  0   0   0  50   0
  0   0   0   0  50

### Step 2: Constructing the X

X is same as in section 3.3, which represents the sex effects.

In [13]:
X = [1 0; 0 1; 0 1; 1 0; 1 0]

5×2 Array{Int64,2}:
 1  0
 0  1
 0  1
 1  0
 1  0

In [14]:
X'*inv(R)*X

2×2 Array{Float64,2}:
 0.065  0.0 
 0.0    0.05

### Step 3: Constructing the W

we know
$
W = 
\begin{pmatrix}
Z\\
Z_1\\
\end{pmatrix}
$, and Z corresponds to the breeding values of the parents animal, which is animal 4,5,6. $Z_1$ corresponds to the breeding value of the non parents animal, which is animal 7,8

In [15]:
Z = [
    0 0 0 1 0 0
    0 0 0 0 1 0
    0 0 0 0 0 1
]

3×6 Array{Int64,2}:
 0  0  0  1  0  0
 0  0  0  0  1  0
 0  0  0  0  0  1

In [16]:
Z1 = [
    0 0 0 0.5 0.5 0 
    0 0 0.5 0 0 0.5
]

2×6 Array{Float64,2}:
 0.0  0.0  0.0  0.5  0.5  0.0
 0.0  0.0  0.5  0.0  0.0  0.5

In [17]:
W = [Z
     Z1]

5×6 Array{Float64,2}:
 0.0  0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0
 0.0  0.0  0.0  0.5  0.5  0.0
 0.0  0.0  0.5  0.0  0.0  0.5

In [18]:
W'* inv(R) * W

6×6 Array{Float64,2}:
 0.0  0.0  0.0    0.0    0.0    0.0  
 0.0  0.0  0.0    0.0    0.0    0.0  
 0.0  0.0  0.005  0.0    0.0    0.005
 0.0  0.0  0.0    0.03   0.005  0.0  
 0.0  0.0  0.0    0.005  0.03   0.0  
 0.0  0.0  0.005  0.0    0.0    0.03 

### Step 4: Constructing the MME

In [37]:
invA = [
     1.833  0.500  0.000 -0.667  0.000 -1.000  
     0.500  2.000  0.500  0.000 -1.000 -1.000 
     0.000  0.500  1.500  0.000 -1.000  0.000  
    -0.667  0.000  0.000  1.333  0.000  0.000 
     0.000 -1.000 -1.000  0.000  2.000  0.000 
    -1.000 -1.000  0.000  0.000  0.000  2.000 
]


α = 40/20

2.0

In [41]:
LHS = [X'*inv(R)*X X'*inv(R)*W
       W'*inv(R)*X W'*inv(R)*W + invA/20]

8×8 Array{Float64,2}:
 0.065  0.0     0.0       0.0     0.01    0.035     0.01    0.01 
 0.0    0.05    0.0       0.0     0.0     0.0       0.025   0.025
 0.0    0.0     0.09165   0.025   0.0    -0.03335   0.0    -0.05 
 0.0    0.0     0.025     0.1     0.025   0.0      -0.05   -0.05 
 0.01   0.0     0.0       0.025   0.08    0.0      -0.05    0.005
 0.035  0.0    -0.03335   0.0     0.0     0.09665   0.005   0.0  
 0.01   0.025   0.0      -0.05   -0.05    0.005     0.13    0.0  
 0.01   0.025  -0.05     -0.05    0.005   0.0       0.0     0.13 

In [43]:
y = data[:WWG]

┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at In[43]:1
└ @ Core In[43]:1


5-element Array{Float64,1}:
 4.5
 2.9
 3.9
 3.5
 5.0

In [46]:
RHS = [
    X'*inv(R)*y
    W'*inv(R)*y
    
]

8-element Array{Float64,1}:
 0.2825             
 0.16999999999999998
 0.0                
 0.0                
 0.05               
 0.14750000000000002
 0.1075             
 0.14750000000000002

In [47]:
sol = inv(LHS)*RHS

8-element Array{Float64,1}:
  4.358476648792877   
  3.4044168159433394  
  0.09848595025687512 
 -0.018768462519237694
 -0.041079363953278936
 -0.00862434894678632 
 -0.18572658819541665 
  0.17689295630873847 

### Step 5: Solve for non parents animals

Remember in section 3.3, we have proved that $a_i = 0.5(a_s + a_d) + m_i$. We can utilize this formula to get the breeding value of non parent animals. Also, for the k term, since the R^{-1} has not been factored out of the MME, $k=\frac{r^{11}}{r^{11} + d^{-1}_i g^{-1}}$.

In section 3.3, the formula for m is $ m_i = k(y - 0.5a_s - 0.5a_d) $, can we use this formula directly in reduced animal model?

In [49]:
k = 0.025/(0.025 + 2*0.05)

0.2

In [50]:
a7 = 0.5*(-0.009 - 0.186) + 0.2*(3.5 - 4.358 - 0.5*(-0.009 - 0.186))

-0.24959999999999993